In [1]:
%pip install -r requirements.txt

  Using cached pandas-2.0.3-cp39-cp39-macosx_10_9_x86_64.whl (11.8 MB)
  Using cached numpy-1.24.4-cp39-cp39-macosx_10_9_x86_64.whl (19.8 MB)
  Using cached pyvis-0.3.2-py3-none-any.whl (756 kB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
     |████████████████████████████████| 7.4 MB 3.7 MB/s eta 0:00:01     |█████                           | 1.1 MB 3.7 MB/s eta 0:00:02     |██████████████████████████████▉ | 7.2 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 34.3 MB/s eta 0:00:01
     |████████████████████████████████| 18.7 MB 423 kB/s eta 0:00:01
  Using cached jsonpickle-4.0.0-py3-none-any.whl (46 kB)
     |████████████████████████████████| 98 kB 18.2 MB/s eta 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached contourpy-1.3.0-cp39-cp39-macosx_10_9_x86_64.whl (265 kB)
  Using cached kiwisolver-1.4.7-cp39-cp39-macosx_10_9_x86_64.whl (65 kB)
  Using cached importlib_resources-6.4.5-py3-none-any.whl (36 kB)
     |█████████████

In [2]:

import pandas as pd
import numpy as np
import pickle
import json
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from datetime import datetime

In [8]:
import fiona
import geopandas as gpd
from shapely.geometry import shape

file_path = "./data/us-states.json"
    
    # Read the GeoJSON data using geopandas
with open(file_path, 'r') as f:
        geojson_data = json.load(f)
features = geojson_data["features"]
    
# Create a list of geometries (Polygons)
geometries = [shape(feature["geometry"]) for feature in features]

# Create a list of state names
state_names = [feature["properties"]["name"] for feature in features]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'state': state_names, 'geometry': geometries})

# Calculate centroids (latitude and longitude)
gdf['centroid'] = gdf.geometry.centroid
gdf['latitude'] = gdf['centroid'].apply(lambda x: x.y)
gdf['longitude'] = gdf['centroid'].apply(lambda x: x.x)

# Extract relevant columns
state_coords = gdf[['state', 'latitude', 'longitude']]
state_coords.to_csv("./data/state_coords.csv", index=False) 
print(state_coords)

                   state   latitude   longitude
0                Alabama  32.789907  -86.827783
1                 Alaska  64.220419 -152.542689
2                Arizona  34.293393 -111.663296
3               Arkansas  34.898249  -92.440920
4             California  37.253895 -119.614389
5               Colorado  38.999340 -105.548773
6            Connecticut  41.620895  -72.727856
7               Delaware  38.982685  -75.497561
8   District of Columbia  38.898378  -77.021809
9                Florida  28.658894  -82.503970
10               Georgia  32.647999  -83.446473
11                Hawaii  20.148899 -156.217626
12                 Idaho  44.388303 -114.659817
13              Illinois  40.062673  -89.195740
14               Indiana  39.912123  -86.270671
15                  Iowa  42.075502  -93.500396
16                Kansas  38.484404  -98.382772
17              Kentucky  37.526932  -85.292111
18             Louisiana  31.061171  -91.988564
19                 Maine  45.378568  -69

In [7]:

# Hand mapped respondent to state
respondent_to_state = {
    'BANC': 'California', 'PSEI': 'California', 'SW': 'Arizona', 'WACM': 'Arizona', 'MISO': 'Michigan', 'SCEG': 'South Carolina',
    'SPA': 'Texas', 'NY': 'New York', 'GVL': 'Georgia', 'FPL': 'Florida', 'PSCO': 'Colorado', 'DUK': 'North Carolina', 
    'ISNE': 'Massachusetts', 'HST': 'Texas', 'DOPD': 'Texas', 'US48': 'North America', 'PJM': 'Pennsylvania', 'AZPS': 'Arizona', 
    'CHPD': 'Texas', 'LDWP': 'California', 'SC': 'South Carolina', 'PNM': 'New Mexico', 'FMPP': 'Florida', 'FLA': 'Florida', 
    'SCL': 'California', 'IID': 'California', 'SWPP': 'Arkansas', 'WAUW': 'Washington', 'TEX': 'Texas', 'MIDA': 'Michigan', 
    'SOCO': 'Georgia', 'NEVP': 'Nevada', 'BPAT': 'Washington', 'ERCO': 'Texas', 'NW': 'Montana', 'CAR': 'North Carolina', 
    'FPC': 'Florida', 'GCPD': 'Texas', 'AECI': 'Missouri', 'PACW': 'California', 'MIDW': 'Wisconsin', 'CPLE': 'Florida', 
    'JEA': 'Florida', 'SRP': 'Arizona', 'PGE': 'California', 'TEN': 'Tennessee', 'CAL': 'California', 'IPCO': 'Oklahoma', 
    'AVA': 'Georgia', 'SEC': 'Texas', 'CISO': 'California', 'LGEE': 'Florida', 'TAL': 'Florida', 'TEC': 'Texas', 
    'NYIS': 'New York', 'TVA': 'Tennessee', 'CPLW': 'Texas', 'TPWR': 'Texas', 'CENT': 'Texas', 'TIDC': 'Texas', 
    'SE': 'Texas', 'WALC': 'Arizona', 'PACE': 'Utah', 'EPE': 'Texas', 'TEPC': 'Texas', 'NWMT': 'Montana', 
    'NE': 'Nebraska'
}

# Load data
data = pd.read_csv("./data/EIA930LoadAndForecast.csv")

data["state"] = data["respondent"].map(respondent_to_state)

# Save the updated dataset
data.to_csv("./data/EIA930LoadAndForecast_with_states.csv", index=False) 

# Data cleaning and transformation
data['value'] = pd.to_numeric(data['value'], errors='coerce')
data['period'] = pd.to_datetime(data['period'])
data = data.dropna().query("period.dt.year >= 2022")
print(data)


        respondent type type_name              period     value  revision_id  \
0             BANC    D    Demand 2022-01-02 02:00:00   2091.00       302352   
1             PSEI    D    Demand 2022-01-02 02:00:00   4437.00       302352   
2               SW    D    Demand 2022-01-02 02:00:00  12142.00       302352   
3             WACM    D    Demand 2022-01-02 02:00:00   3212.00       302352   
4             MISO    D    Demand 2022-01-02 02:00:00  74428.00       302352   
...            ...  ...       ...                 ...       ...          ...   
7715101        TEN    D    Demand 2024-07-04 06:00:00  20502.36       579043   
7715102        GVL    D    Demand 2024-07-04 06:00:00    256.00       579043   
7715103       PACW    D    Demand 2024-07-04 06:00:00   2495.00       579043   
7715104       NEVP    D    Demand 2024-07-04 06:00:00   6843.00       579043   
7715105       AZPS    D    Demand 2024-07-04 06:00:00   6251.00       579043   

               id       state  
0      

In [9]:

# Mark anomalies
def mark_anomalies(data):
    data['is_zero'] = (data['value'] == 0).astype(int)
    data['is_negative'] = (data['value'] < 0).astype(int)
    
    data['is_spike'] = data.groupby(['respondent', 'type_name'])['value'].transform(lambda x: (x > x.quantile(0.999)).astype(int))
    data['is_spike'] = data['is_spike'].fillna(0)
    return data

# Impute data
def impute_data(data):
    # Mark impute column
    data['impute'] = (data['is_zero'] + data['is_negative'] + data['is_spike'] > 0).astype(int)
    
    # Split into actuals and forecast
    actuals = data[data['type_name'] == "Demand"]
    forecast = data[data['type_name'] == "Day-ahead demand forecast"]
    
    # Merge actuals with forecast
    joined = pd.merge(
        actuals,
        forecast.rename(columns={'value': 'forecast'}),
        on=['respondent', 'period'],
        how='left'
    )
    
    # Rename columns to avoid suffixes like `_x` and `_y`
    joined = joined.rename(columns={
        'impute_x': 'impute',
        'type_x': 'type',
        'type_name_x': 'type_name',
    })
    
    # Add imputed values
    joined['imputed'] = np.where(
        (joined['impute'] == 1) & ~joined['forecast'].isna(),
        joined['forecast'],
        np.where(
            (joined['impute'] == 1) & joined['forecast'].isna() & ~joined['forecast'].shift(1).isna(),
            joined['forecast'].shift(1),
            np.where(
                (joined['impute'] == 1) & joined['forecast'].isna() & joined['forecast'].shift(1).isna(),
                joined['value'].shift(1),
                joined['value']
            )
        )
    )
    
    # Return cleaned data
    return joined[['respondent', 'period', 'type', 'type_name', 'imputed']].rename(
        columns={'imputed': 'value'}
    ).drop_duplicates()
data_marked = mark_anomalies(data)
data_imputed = impute_data(data_marked)
data_imputed.to_csv("./data/data_imputed.csv", index=False)

for i in range(5):
    print(f"{i+1} of 5")
    data_marked = mark_anomalies(data_marked)
    data_imputed = impute_data(data_marked)

raw_imputed = pd.merge(data, data_imputed.rename(columns={'value': 'imputed'}),
                       on=['respondent', 'type', 'type_name', 'period'], how='left')
raw_imputed['is_imputed'] = (raw_imputed['value'] != raw_imputed['imputed']).astype(int)
raw_imputed.to_csv("./data/raw_imputed.csv", index=False)

print(raw_imputed['is_imputed'].sum())
# Calculate MAPEs
actuals = raw_imputed[raw_imputed['type_name'] == "Demand"]
forecast = raw_imputed[raw_imputed['type_name'] == "Day-ahead demand forecast"]
joined = pd.merge(actuals, forecast[['respondent', 'period', 'value']].rename(columns={'value': 'forecast'}),
                  on=['respondent', 'period'], how='left')
joined['abs_error'] = np.abs(joined['value'] - joined['forecast']) / np.abs(joined['value'])

MAPE = joined[joined['abs_error'] != np.inf].groupby('respondent')['abs_error'].mean().reset_index(name='MAPE')
MAPE.to_csv("./data/MAPE.csv", index=False)

# Load edges and calculate correlations
edges = pd.read_csv("./data/eia_930_edges.csv")
exclude = ["CISO", "ERCO", "SWPP", "MISO", "NYIS", "ISNE", "CAL", "PJM"]

edges = edges.merge(MAPE, left_on="node1", right_on="respondent").rename(columns={"MAPE": "MAPE_node1"})
edges = edges.merge(MAPE, left_on="node2", right_on="respondent").rename(columns={"MAPE": "MAPE_node2"})
edges['abs_diff'] = np.abs(edges['MAPE_node1'] - edges['MAPE_node2'])
edges = edges.query("~node1.isin(@exclude) & ~node2.isin(@exclude)").sort_values('abs_diff', ascending=False)
edges.to_csv("./data/edges_with_MAPE.csv", index=False)

# Wide format and correlation matrix
duplicates = actuals[actuals.duplicated(subset=['period', 'respondent'], keep=False)]
if not duplicates.empty:
    print("Duplicates found in actuals before pivot:")
    print(duplicates)
    # dedup
    actuals = actuals.drop_duplicates(subset=['period', 'respondent'])

# Perform pivot operation
actuals_wide = actuals.pivot(index='period', columns='respondent', values='imputed')
correlation_matrix = actuals_wide.corr(method='pearson', min_periods=1)
correlation_matrix.to_csv("./data/correlation_matrix.csv")

# Simple LDWP Model
relevant_cols = ['CISO', 'BPAT', 'LDWP', 'PACE', 'NEVP', 'AZPS', 'WALC']
reg_data = actuals_wide[relevant_cols].dropna()
reg_data['LDWP_lag1'] = reg_data['LDWP'].shift(1)
reg_data['LDWP_lag24'] = reg_data['LDWP'].shift(24)

reg_data = reg_data.dropna()
X = reg_data[['LDWP_lag1', 'LDWP_lag24', 'CISO', 'BPAT', 'PACE', 'NEVP', 'AZPS', 'WALC']]
y = reg_data['LDWP']



1 of 5
2 of 5
3 of 5
4 of 5
5 of 5
1471658
Duplicates found in actuals before pivot:
        respondent type type_name              period    value  revision_id  \
88              NE    D    Demand 2022-01-01 00:00:00  14859.0       302352   
89             PNM    D    Demand 2022-01-01 00:00:00   1754.0       302352   
90            LDWP    D    Demand 2022-01-01 00:00:00   2662.0       302352   
91            BPAT    D    Demand 2022-01-01 00:00:00   8535.0       302352   
92            CISO    D    Demand 2022-01-01 00:00:00  22618.0       302352   
...            ...  ...       ...                 ...      ...          ...   
2942730       BANC    D    Demand 2024-07-05 06:00:00   3115.0       579043   
2942731       CPLW    D    Demand 2024-07-05 06:00:00    504.0       579043   
2942732       TIDC    D    Demand 2024-07-05 06:00:00    520.0       579043   
2942733        DUK    D    Demand 2024-07-05 06:00:00  12678.0       579043   
2942734        TAL    D    Demand 2024-07-05 0

In [37]:
# Define hyperparameter grids for GridSearchCV
linear_param_grid = {
    'fit_intercept': [True, False]
}

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'random_state': [42]
}

gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 10],
    'random_state': [42]
}

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, shuffle=True, random_state=614
        )

# Model 1: Linear Regression with GridSearchCV and Cross-Validation
linear_model = LinearRegression()
linear_grid_search = GridSearchCV(linear_model, linear_param_grid, cv=5, scoring='neg_mean_absolute_error')
linear_grid_search.fit(X_train, y_train)

# Perform cross-validation for Linear Regression
linear_cv_results = cross_validate(linear_grid_search.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
linear_mean_score = linear_cv_results['test_score'].mean()

# Best Linear Regression Model
best_linear_model = linear_grid_search.best_estimator_
linear_predictions = best_linear_model.predict(X_test)
linear_mape = mean_absolute_percentage_error(y_test, linear_predictions)
with open("./models/linear_regression_model.pkl", 'wb') as f:
    pickle.dump(best_linear_model, f)
print("Best Linear Regression Model saved successfully.")

# Model 2: Random Forest Regressor with GridSearchCV and Cross-Validation
rf_model = RandomForestRegressor(random_state=42)
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=5, scoring='neg_mean_absolute_error')
rf_grid_search.fit(X_train, y_train)

# Perform cross-validation for Random Forest
rf_cv_results = cross_validate(rf_grid_search.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
rf_mean_score = rf_cv_results['test_score'].mean()

# Best Random Forest Model
best_rf_model = rf_grid_search.best_estimator_
rf_predictions = best_rf_model.predict(X_test)
rf_mape = mean_absolute_percentage_error(y_test, rf_predictions)
with open("./models/random_forest_model.pkl", 'wb') as f:
    pickle.dump(best_rf_model, f)
print("Best Random Forest Model saved successfully.")

# Model 3: Gradient Boosting Regressor with GridSearchCV and Cross-Validation
gb_model = GradientBoostingRegressor(random_state=42)
gb_grid_search = GridSearchCV(gb_model, gb_param_grid, cv=5, scoring='neg_mean_absolute_error')
gb_grid_search.fit(X_train, y_train)

# Perform cross-validation for Gradient Boosting
gb_cv_results = cross_validate(gb_grid_search.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
gb_mean_score = gb_cv_results['test_score'].mean()

# Best Gradient Boosting Model
best_gb_model = gb_grid_search.best_estimator_
gb_predictions = best_gb_model.predict(X_test)
gb_mape = mean_absolute_percentage_error(y_test, gb_predictions)
with open("./models/gradient_boosting_model.pkl", 'wb') as f:
    pickle.dump(best_gb_model, f)
print("Best Gradient Boosting Model saved successfully.")

# Compare model accuracy
print("Model Performance (MAPE):")
print(f"Linear Regression: {linear_mape:.4f}, Cross-Validation Mean: {linear_mean_score:.4f}")
print(f"Random Forest Regressor: {rf_mape:.4f}, Cross-Validation Mean: {rf_mean_score:.4f}")
print(f"Gradient Boosting Regressor: {gb_mape:.4f}, Cross-Validation Mean: {gb_mean_score:.4f}")

# Evaluation results dictionary
evaluation_results = {
    "Linear Regression": {"MAPE": linear_mape, "Cross-Validation Mean": linear_mean_score},
    "Random Forest": {"MAPE": rf_mape, "Cross-Validation Mean": rf_mean_score},
    "Gradient Boosting": {"MAPE": gb_mape, "Cross-Validation Mean": gb_mean_score},
}

# Save evaluation results to JSON
with open("./data/evaluation_results.json", "w") as file:
    json.dump(evaluation_results, file)

print("Evaluation results saved successfully.")

# Print model coefficients or feature importances
print("Linear Model Coefficients:", best_linear_model.coef_)
print("Random Forest Feature Importances:", best_rf_model.feature_importances_)
print("Gradient Boosting Feature Importances:", best_gb_model.feature_importances_)

print("Results saved successfully.")

Best Linear Regression Model saved successfully.
Best Random Forest Model saved successfully.
Best Gradient Boosting Model saved successfully.
Model Performance (MAPE):
Linear Regression: 0.1275, Cross-Validation Mean: -103.7426
Random Forest Regressor: 0.4140, Cross-Validation Mean: -98.6381
Gradient Boosting Regressor: 0.6436, Cross-Validation Mean: -98.4495
Evaluation results saved successfully.
Linear Model Coefficients: [ 0.79309392  0.16407204 -0.00480436  0.0096736   0.06658138 -0.02581321
  0.0076787   0.08533339]
Random Forest Feature Importances: [0.95277565 0.01765398 0.01075002 0.00460675 0.00767984 0.00185537
 0.00171825 0.00296015]
Gradient Boosting Feature Importances: [9.50294019e-01 2.55986956e-02 9.58802399e-03 4.14949238e-03
 6.83537775e-03 7.41416470e-04 8.79949126e-04 1.91302604e-03]
Results saved successfully.
